<a href="https://colab.research.google.com/github/glaunay/four_point_five/blob/main/master_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TO RP/GLA 15/11

### Requirements
1. docking "trimer"
2. linky exec
3. polymer library file with building blocks:
  * atom name sequence
  * average distance between blocks
4. python postprocessing
5. IMP refiner wrapper

### step 1 / link_generator
linky complex.pdb PEG

### output as json format
```json
{
  "atom_index" : [
    (x, y, z, M, is_opt:bool),
  ...
  ]
}
```

Where,
- coordinates sequence respect the one of the atoms polymer, making dynamic bond term declaration trivial
- x,y,z values of all atoms of a given building block (eg:PEG_1)
- 1st atom of sequence is last of Lig/warhead domain
- last atom of sequence is the 1st of Warhead/lig domain

## step 2 / IMP preprocessing
### Generate particles
### Define softsphere potentials
### Define dihedrals

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c salilab imp
!conda install matplotlib

In [ ]:
try:
  del m
  del mc
  del rs
  del hroot
  del mvs
  del rh
except:
  print("all good")

import IMP
import IMP.atom
import IMP.core

all good


In [ ]:
# create an IMP model
m=IMP.Model()

In [ ]:
#each tuple is x,y,z,radius,mass,is_optimized
particles=[(0,0,0,1,100,False),
 (0,0,2,1,100,True),
 (0,0,4,1,100,True),
 (0,0,6,1,100,True),
 (0,0,8,1,100,True),
 (0,0,10,1,100,True),
 (0,0,12,1,100,False),
 (0,0,7,1,100,False)]

In [ ]:
def setup_representation(tpl):
    # decorate it as a sphere
    pa=IMP.Particle(m)
    dr=IMP.core.XYZR.setup_particle(pa)

    # set the coordinates
    dr.set_coordinates((tpl[0],tpl[1],tpl[2]))

    # set the radius
    dr.set_radius(tpl[3])

    # set the mass
    IMP.atom.Mass.setup_particle(pa,tpl[4])

    # set the optimization of the coordinates to True
    dr.set_coordinates_are_optimized(tpl[5])

    ha=IMP.atom.Hierarchy(pa)
    cb=IMP.display.Color(0,1,1)
    c=IMP.display.Colored.setup_particle(pa,cb)

    return ha

In [ ]:
def setup_movers(particles):
    return_list=[]
    for p in particles:
        dr=IMP.core.XYZR(p)
        if dr.get_coordinates_are_optimized():
          mv=IMP.core.BallMover(m,p,0.1)
          return_list.append(mv)
    return return_list


In [ ]:
def setup_restraints(particles):
    return_list=[]
    for i in range(len(particles) - 1):
      p0 = particles[i]
      p1 = particles[i + 1]
      # create a harmonic restraint
      dr0=IMP.core.XYZR(p0)
      dr1=IMP.core.XYZR(p1)
      if not dr0.get_coordinates_are_optimized() and not dr1.get_coordinates_are_optimized():
        continue
      hf = IMP.core.Harmonic(0.1,10.0)
      dr=IMP.core.DistanceRestraint(m,hf,p0,p1)
      return_list.append(dr)
    return return_list

In [ ]:
def setup_exvol(particles):
    ssps = IMP.core.SoftSpherePairScore(10.0)
    lsa = IMP.container.ListSingletonContainer(m)
    #print(IMP.get_indexes(particles))
    lsa.add(IMP.get_indexes(particles))
    rbcpf = IMP.core.RigidClosePairsFinder()
    cpc = IMP.container.ClosePairContainer(lsa,0.0,rbcpf,10.0)
    #print(cpc.get_indexes())
    #print(rbcpf.get_close_pairs(m,IMP.get_indexes(particles)))
    evr = IMP.container.PairsRestraint(ssps,cpc)
    return evr

In [ ]:
hs=[setup_representation(t) for t in particles ]

In [ ]:
mvs=setup_movers(hs)

In [ ]:
rs=setup_restraints(hs)

In [ ]:
evr=setup_exvol(hs)
print(evr.unprotected_evaluate(None))
rs.append(evr)

0.0


In [ ]:
# wrap the restraints in a Scoring Function
sf = IMP.core.RestraintsScoringFunction(rs)

In [ ]:
# Build the Monte Carlo Sampler
mc = IMP.core.MonteCarlo(m)
mc.set_scoring_function(sf)
sm = IMP.core.SerialMover(mvs)
mc.add_mover(sm)
mc.set_return_best(False)
mc.set_kt(1.0)

In [ ]:


# Prepare the trajectory file
import IMP.rmf
import RMF

hroot=IMP.atom.Hierarchy(IMP.Particle(m))

for h in hs: hroot.add_child(h)

rh = RMF.create_rmf_file("out.rmf")
IMP.rmf.add_hierarchies(rh, [hroot])
IMP.rmf.add_restraints(rh,rs)
IMP.rmf.save_frame(rh)

f0

In [ ]:
# run the sampling
for i in range(10000):
    mc.optimize(1)
    #if i%10==0:
    #  IMP.rmf.save_frame(rh)
    #  #print(sf.evaluate(False),evr.unprotected_evaluate(None))

cg=IMP.core.ConjugateGradients(m)
cg.set_scoring_function(rs)
cg.optimize(100)
IMP.rmf.save_frame(rh)
del rh